In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
from keras.layers import Input, Lambda, Dense, Flatten

In [4]:
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [5]:
IMAGE_SIZE = [224, 224]

train_path = '/gdrive/My Drive/input/labeled-chest-xray-images/chest_xray/train/'
test_path = '/gdrive/My Drive/input/labeled-chest-xray-images/chest_xray/test/'

In [6]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58900480/58889256 [==============================] - 0s 0us/step


In [22]:
for layer in vgg.layers:
    layer.trainable = False

In [23]:
folders = glob('/gdrive/My Drive/input/labeled-chest-xray-images/chest_xray/train/*')
x = Flatten() (vgg.output)

In [24]:
prediction = Dense(len(folders), activation='softmax') (x)
# creating a model object
model = Model(inputs=vgg.input, outputs=prediction)
# view structure of the model
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [25]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [26]:
from keras.preprocessing.image import ImageDataGenerator

In [14]:
# note: must input code received from google for access to the google drive files: Please copy this code, switch to your application and paste it there:
# 4/1AX4XfWgMbu2qOfhAjCXU90DNP_Yx7EFgJsSBJrHiKfPqnwomhdAgQZxkp3o
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
# ('/input/labeled-chest-xray-images/chest_xray/train')

In [27]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.3,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)




# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/gdrive/My Drive/input/labeled-chest-xray-images/chest_xray/train/',
                                                 target_size = (224, 224),
                                                 batch_size = 10,
                                                 class_mode = 'categorical')




test_set = test_datagen.flow_from_directory('/gdrive/My Drive/input/labeled-chest-xray-images/chest_xray/test/',
                                            target_size = (224, 224),
                                            batch_size = 10,
                                            class_mode = 'categorical')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [29]:
r = model.fit_generator(   
  training_set,
  validation_data=test_set,
  epochs=3,
  steps_per_epoch=len(training_set)/5,
  validation_steps=len(test_set)/5,
  workers=-1,
)

Epoch 1/3
104/104 [==============================] - 935s 9s/step - loss: 0.3014 - accuracy: 0.8752 - val_loss: 0.3058 - val_accuracy: 0.8846
Epoch 2/3
104/104 [==============================] - 841s 8s/step - loss: 0.1854 - accuracy: 0.9238 - val_loss: 0.5192 - val_accuracy: 0.8692
Epoch 3/3
104/104 [==============================] - 785s 8s/step - loss: 0.2248 - accuracy: 0.9305 - val_loss: 0.2072 - val_accuracy: 0.9154


In [30]:
import tensorflow as tf
from keras.models import load_model
# from google.colab import drive
# drive.mount()
# %cd /gdrive
model.save('/gdrive/My Drive/input/chest_xray_testing_only.h5')

In [31]:
from keras.models import load_model

In [32]:
from keras.preprocessing import image

In [33]:
from keras.applications.vgg16 import preprocess_input

In [34]:
import numpy as np

In [35]:
from keras.models import load_model
model=load_model('/gdrive/My Drive/input/chest_xray_testing_only.h5')

In [36]:
model.compile(metrics=[tf.keras.metrics.Recall()])

In [37]:
model.evaluate(test_set, return_dict=True, use_multiprocessing=True, workers=-1)

63/63 [==============================] - 384s 6s/step - loss: 0.0000e+00 - recall: 0.8958


{'loss': 0.0, 'recall': 0.8958333134651184}

In [38]:
#conf_matrix = tf.math.confusion_matrix(labels=test_set,
predictions=model.predict(test_set, workers=-1)
predictions

array([[4.2186341e-05, 9.9995780e-01],
       [9.9779439e-01, 2.2056031e-03],
       [3.1464370e-03, 9.9685353e-01],
       ...,
       [2.2705994e-03, 9.9772936e-01],
       [7.4862200e-04, 9.9925131e-01],
       [9.9345809e-01, 6.5419702e-03]], dtype=float32)

In [39]:
test_set.classes[190:]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [41]:
rnd_predictions = [round(pred[0]) for pred in predictions]
np.array(rnd_predictions)[190:]

array([1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0,
       1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,

In [42]:
conf_matrix = tf.math.confusion_matrix(labels=test_set.classes, predictions=np.array(rnd_predictions))
conf_matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[162,  72],
       [249, 141]], dtype=int32)>

In [43]:
img=image.load_img('/gdrive/My Drive/input/labeled-chest-xray-images/chest_xray/test/PNEUMONIA/person1_virus_6.jpeg',target_size=(224,224))

In [44]:
x=image.img_to_array(img)

In [45]:
x=np.expand_dims(x, axis=0)

In [46]:
img_data=preprocess_input(x)

In [47]:
classes=model.predict(img_data)

In [48]:
# classes
result=int(classes[0][0])

In [49]:
if result==0:
  print("X-Ray results indicate Pneumonia")
else:
  print("X-Ray results are Normal")
  

X-Ray results indicate Pneumonia


In [50]:
result

0